In [1]:
import gym
import itertools
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

import baselines.common.tf_util as U

from baselines import logger
from baselines import deepq
from baselines.deepq.replay_buffer import PrioritizedReplayBuffer
from baselines.common.schedules import LinearSchedule

In [2]:

import gym_minecraft
from MinecraftGym import MinecraftWrapper

In [3]:
from keras import backend as K
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation,GRU,Input,LSTM,Conv2D,Flatten
from keras.optimizers import RMSprop,Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [4]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.driving import linear
from bokeh.layouts import row,gridplot
from IPython.display import clear_output,display
output_notebook()

Loading BokehJS ...

In [5]:
from hyperdash import Experiment

In [6]:
pre_env = gym.make("MinecraftCliffWalking1-v0")
pre_env.init(videoResolution=[400,400],allowContinuousMovement=["move", "turn", "strafe"],observeGrid=[20,-1,20,20,-1,20],observeDistance=[4,45,12])
env = MinecraftWrapper(pre_env,1/10,(41,41))

In [7]:
def proc2_reward(info):
    if info is None:
        return 0
    elif 'observation' not in info.keys():
        return 0
    elif info['observation'] is None:
        return 0
    elif 'distanceFromdist' in info['observation'].keys():
        return 10/(0.001 + info['observation']['distanceFromdist'])
    else:
        return 0

In [8]:
def model(inpt, num_actions, scope, reuse=False):
    """This model takes as input an observation and returns values of all actions."""
    with tf.variable_scope(scope, reuse=reuse):
        model = Sequential()
        model.add(Conv2D(32,(8,8),input_shape=env.observation_space.shape,activation='relu'))
        model.add(Conv2D(64,(4,4),activation='relu'))
        model.add(Conv2D(64,(3,3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(60,activation='relu'))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(output_dim=num_actions,activation='softmax'))
        
        '''
        a = Conv2D(32,(8,8),input_shape=env.observation_space.shape,activation='relu')(inpt)
        b = Conv2D(64,(3,3),activation='relu')(a)
        d = Flatten()(b)
        c = Dense(output_dim=num_actions,activation='softmax')(d)
        '''
        return model.output

In [9]:
def old_model(inpt, num_actions, scope, reuse=False):
    """This model takes as input an observation and returns values of all actions."""
    with tf.variable_scope(scope, reuse=reuse):
        print(inpt.shape)
        out = inpt

        out =  tf.layers.conv2d(
                inputs=out,
                filters=32,
                kernel_size=[8, 8],
                padding="same",
                activation=tf.nn.relu)
        out =  tf.layers.conv2d(
                inputs=out,
                filters=64,
                kernel_size=[4, 4],
                padding="same",
                activation=tf.nn.relu)
        out =  tf.layers.conv2d(
                inputs=out,
                filters=64,
                kernel_size=[5, 5],
                padding="same",
                activation=tf.nn.relu)
        out2 = tf.contrib.layers.flatten(out)
        out2 = tf.layers.dense(inputs=out2, units=64)
        out2 = tf.layers.dense(inputs=out2, units=num_actions)
    return out2

In [14]:
U.reset()
exp  = Experiment("openAI DQN")
with U.make_session(2) as sess:
    # Create all the functions necessary to train the model
    K.set_session(sess)

    act, train, update_target, debug = deepq.build_train(
        make_obs_ph=lambda name: U.BatchInput((40,40,1), name=name),
        q_func=old_model,
        num_actions=env.action_space.n,
        optimizer=tf.train.AdamOptimizer(learning_rate=5e-4),
    )
    
    # Create the replay buffer
    replay_buffer = PrioritizedReplayBuffer(50000,0.9)
    # Create the schedule for exploration starting from 1 (every action is random) down to
    # 0.02 (98% of actions are selected according to values predicted by the model).
    exploration = LinearSchedule(schedule_timesteps=10000, initial_p=1.0, final_p=0.02)

    # Initialize the parameters and copy them to the target network.
    U.initialize()
    update_target()

    episode_rewards = [0.0]
    obs = env.reset()
    for t in itertools.count():
        # Take action and update exploration to the newest value
        action = act(obs[None], update_eps=exploration.value(t))[0]
        new_obs, rew, done, info = env.step(action)
        rew += proc2_reward(info)
        # Store transition in the replay buffer.
        replay_buffer.add(obs, action, rew, new_obs, float(done))
        obs = new_obs

        episode_rewards[-1] += rew
        if done:
            obs = env.reset()
            exp.metric("reward",episode_rewards[-1])
            episode_rewards.append(0)

        is_solved = t > 100 and np.mean(episode_rewards[-101:-1]) >= 200
        if is_solved:
            # Show off the result
            #env.render()
            print("Solved")
        else:
            # Minimize the error in Bellman's equation on a batch sampled from replay buffer.
            if t > 1000:
                obses_t, actions, rewards, obses_tp1, dones, _,_ = replay_buffer.sample(32,0.9)
                train(obses_t, actions, rewards, obses_tp1, dones, np.ones_like(rewards))
            # Update target network periodically.
            if t % 1000 == 0:
                update_target()

        if done and len(episode_rewards) % 10 == 0:
            logger.record_tabular("steps", t)
            logger.record_tabular("episodes", len(episode_rewards))
            logger.record_tabular("mean episode reward", round(np.mean(episode_rewards[-101:-1]), 1))
            logger.record_tabular("% time spent exploring", int(100 * exploration.value(t)))
            logger.dump_tabular()

Instructions for updating:
Use `argmax` instead
(?, 40, 40, 1)
(?, 40, 40, 1)
(?, 40, 40, 1)
(?, 40, 40, 1)
| reward: -95.277886 |
| reward: -96.205312 |
| reward: -95.248804 |
| reward: -96.217131 |
| reward: -96.222051 |
| reward: -96.210500 |
| reward: -95.247247 |
| reward: -93.389679 |
| reward: -96.212986 |
-------------------------------------
| % time spent exploring | 99       |
| episodes               | 10       |
| mean episode reward    | -95.6    |
| steps                  | 50       |
-------------------------------------
| reward: -96.214437 |
| reward: -95.282010 |
| reward: -86.660289 |
| reward: -96.217131 |
| reward: -95.248688 |
| reward: -95.256792 |
| reward: -95.248418 |
| reward: -94.289390 |
| reward: -96.211992 |
| reward: -90.522791 |
-------------------------------------
| % time spent exploring | 98       |
| episodes               | 20       |
| mean episode reward    | -94.8    |
| steps                  | 122      |
-------------------------------------

Agent missed 1 observation(s).


| reward: -93.391489 |


Agent missed 2 observation(s).


ResourceExhaustedError: OOM when allocating tensor with shape[32,32,40,40]
	 [[Node: deepq_1/gradients/deepq_1/q_func/conv2d_2/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](deepq_1/gradients/deepq_1/q_func/conv2d_2/convolution_grad/Shape, deepq/q_func/conv2d_1/kernel/read, deepq_1/gradients/deepq_1/q_func/conv2d_2/BiasAdd_grad/tuple/control_dependency)]]

Caused by op 'deepq_1/gradients/deepq_1/q_func/conv2d_2/convolution_grad/Conv2DBackpropInput', defined at:
  File "/home/ubuntu/miniconda2/envs/casper/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/miniconda2/envs/casper/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/miniconda2/envs/casper/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_ne

In [13]:
exp.end()

In [ ]:
        Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        """
        
                Returns
        -------
        obs_batch: np.array
            batch of observations
        act_batch: np.array
            batch of actions executed given obs_batch
        rew_batch: np.array
            rewards received as results of executing act_batch
        next_obs_batch: np.array
            next set of observations seen after executing act_batch
        done_mask: np.array
            done_mask[i] = 1 if executing act_batch[i] resulted in
            the end of an episode and 0 otherwise.
        weights: np.array
            Array of shape (batch_size,) and dtype np.float32
            denoting importance weight of each sampled transition
        idxes: np.array
            Array of shape (batch_size,) and dtype np.int32
            idexes in buffer of sampled experiences
        """

# Tests

In [ ]:
def reward_proc(info,grid_shape = (41,41)):
    if "grid" not in info['observation'].keys():
        return 0
    # reformat grid to a vector that only show the floor with blocks
    vec = []    
    for item in info['observation']['grid'][::-1]:
        if 'lava' in item:
            vec.append(1)
        elif 'lapis' in item:
            vec.append(2)
        else:
            vec.append(0)

     # compute reward depending on distance to target
    new_state = np.array(vec)
    tmp = np.array(vec).reshape(grid_shape)
    idx2 = np.argwhere(tmp == 2)

    size = grid_shape[0]
    idx1 = (np.ceil(size/2),np.ceil(size/2))

    a = (self._dist(idx2,idx1))
    if(a > 0):
        try:
            dist_reward = 2000/(a)
        except:
            dist_reward = 0
        return dist_reward

In [ ]:
def update(x,y,handle,plot):
    plot.data_source.data['x'] += [x]
    plot.data_source.data['y'] += [y]
    push_notebook(handle=handle)

In [ ]:
fig_test = figure(plot_width=1000, plot_height=400,title="rewards_test",
                      x_axis_label="x",
                      y_axis_label="y")
test_plot = fig_test.line([],[],color="navy",line_width=2)
# make a grid
handle_test = show(fig_test, notebook_handle=True)

In [ ]:
count = 0
for e in range(100):
    done = False
    obs = env.reset()
    #env.render()
    count = 0
    while not done:
        
        #env.render()
        a = env.action_space.sample()
        s,r,done,info = env.step(a)
        r_ = proc2_reward(info)
        print(r_)
        r += r_
        update(count,r_,handle_test,test_plot)
        count += 1
    

In [ ]:
env.close()

In [ ]:
env.reset()
s,r,done,info = env.step(0)

In [ ]:
s,r,done,info = env.step(0)
info

In [ ]:
proc2_reward(info)